In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
os.environ["CUDA_VISIBLE_DEVICES"] = '3'

In [2]:
# adv_trained_model_path = '/workdir/security/home/junjiehuang2468/paper/trained_models_weight/ember/adversarial_training/append_FSM/'
# adv_best_trained_model = '2022-01-25 15:05/append_FSM_epoch:1_test_acc:0.880683.pt'

In [3]:
adv_trained_model_path = '/workdir/security/home/junjiehuang2468/paper/trained_models_weight/ember/adversarial_training/PGD_2/'
adv_best_trained_model = '2022-01-26 09:38/PGD_epoch:1_test_acc:0.878075.pt'

In [4]:
trained_model_path = '/workdir/security/home/junjiehuang2468/paper/trained_models_weight/ember/'
best_trained_model = '2022-01-18 14:55/2w_epoch:0_test_acc:0.890858.pt'
data_path = "/workdir/security/home/junjiehuang2468/paper/data/ember2018/"
train_data_path = data_path + "malwares/" 
test_data_path = data_path + "test_malwares/" 

In [5]:
CUDA = True if torch.cuda.is_available() else False
NUM_WORKERS = 24
BATCH_SIZE = 20
# BATCH_SIZE = 10
LEAVE_BIT_NUMBER = 20000
KERNEL_SIZE = 500

In [6]:
LEAVE_BIT_NUMBER_DICT = {10000:'1w',20000:'2w'}

In [7]:
trainset = pd.read_csv(data_path + 'train_dataset.csv')
validset = pd.read_csv(data_path + 'valid_dataset.csv')
testset = pd.read_csv(data_path + 'test_dataset.csv')
testset = testset.iloc[np.argwhere(testset['labels'].values == 1).squeeze(),:]

In [8]:
class ExeDataset(Dataset):
    def __init__(self, malware_names, data_path, labels, leave_bit_num):
        self.malware_names = malware_names
        self.data_path = data_path
        self.labels = labels
        self.leave_bit_num = leave_bit_num

    def __len__(self):
        return len(self.malware_names)

    def __getitem__(self, idx):
        with open(self.data_path + self.malware_names[idx] + '.txt','rb') as fp:
            data = [bit+1 for bit in fp.read()[:self.leave_bit_num]]
            padding = [0]*(self.leave_bit_num-len(data))
            data = data + padding

        return np.array(data), np.array([self.labels[idx]])

In [9]:
train_dataset = ExeDataset(
    trainset["id"].tolist(), 
    train_data_path, 
    trainset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
valid_dataset = ExeDataset(
    validset["id"].tolist(), 
    train_data_path, 
    validset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
test_dataset = ExeDataset(
    testset["id"].tolist(), 
    test_data_path, 
    testset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)

In [10]:
trainloader = DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
validloader = DataLoader(
    dataset = valid_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
testloader = DataLoader(
    dataset = test_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)

In [17]:
class Model(nn.Module):
    def __init__(self, data_length = 2e6, kernel_size = 500):
        super().__init__()
        self.embedding = nn.Embedding(257, 8, padding_idx=0)
        self.conv_layer_1 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        # self.bn_1 = nn.BatchNorm1d(128)
        self.conv_layer_2 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        self.pool_layer_2 = nn.MaxPool1d(data_length//kernel_size)
        self.fc_layer_3 = nn.Linear(128, 128)
        self.fc_layer_4 = nn.Linear(128, 2)
        
    def sequence(self,embedd_x):
        x = embedd_x.transpose(-1,-2)
        x_conv_1 = self.conv_layer_1(x[:,:4,:])
        x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))
        x = x_conv_1*x_conv_2
        del x_conv_1,x_conv_2
        x = self.pool_layer_2(x).squeeze()
        x = self.fc_layer_3(x)
        x = self.fc_layer_4(x)
        return x
        
    def forward(self, input_, loss_fn, optim, true_label):
        batch_acc = []
        batch_grad = []
        
        embedd_x = self.embedding(input_)
        x = self.sequence(embedd_x)
        acc = torch.argmax(torch.softmax(x,dim=-1),dim=-1).float().mean()
        batch_acc.append(acc.tolist())
        
        padding_place_mask = input_ == 0
        random_padding = torch.randint_like(input=padding_place_mask.float(),low=1,high=257) * padding_place_mask.float()
        input_ += random_padding.long()
        
        one_hot_x = F.one_hot(input_,num_classes=257).float()
        one_hot_x.requires_grad = True
        one_hot_x.retain_grad()
        for _ in range(100):
            embedd_x = one_hot_x @ self.embedding.weight
            x = self.sequence(embedd_x)
            
            acc = torch.argmax(torch.softmax(x,dim=-1),dim=-1).float().mean()
            batch_acc.append(acc.tolist())
            
            mislead_labels = torch.abs(true_label - 1)
            
            optim.zero_grad()
            loss = loss_fn(x,mislead_labels) - torch.pow(one_hot_x,torch.inf).mean()
            loss.backward()
            optim.step()
            
            shape = padding_place_mask.shape
            expand_padding_place_mask = padding_place_mask.unsqueeze(-1).expand(shape[0],shape[1],257)
            temp_one_hot_x = F.one_hot(input_,num_classes=257).float()
            one_hot_x.data = temp_one_hot_x*(1 - expand_padding_place_mask.float()) + one_hot_x*expand_padding_place_mask.float()
            
        embedd_x = one_hot_x @ self.embedding.weight
        x = self.sequence(embedd_x)
        return x.detach().cpu().numpy(),batch_acc,batch_grad,one_hot_x

In [18]:
model = Model(data_length=LEAVE_BIT_NUMBER,kernel_size=KERNEL_SIZE)

ce_loss = nn.CrossEntropyLoss()
optim = Adam(model.parameters())

model = model.cuda() if CUDA else model
ce_loss = ce_loss.cuda() if CUDA else ce_less

In [19]:
# model.load_state_dict(torch.load(trained_model_path + best_trained_model))
model.load_state_dict(torch.load(adv_trained_model_path + adv_best_trained_model))

<All keys matched successfully>

In [ ]:
model.eval()
acc = []
preds = []
labels = []
total_batch_acc = []
total_batch_grad = []
bar = tqdm(testloader)
# bar = tqdm(validloader)
for step, (batch_data,batch_label) in enumerate(bar):
    batch_data = batch_data.cuda() if CUDA else batch_data
    batch_label = batch_label.cuda() if CUDA else batch_label
    batch_label = batch_label.squeeze()
    true_label = torch.zeros((len(batch_label),2))
    for idx,target in enumerate(batch_label.squeeze()): true_label[idx,target] = 1
    true_label = true_label.cuda() if CUDA else true_label

#     pred,input_,one_hot_x = model(batch_data,ce_loss)
#     break
    pred,batch_acc,batch_grad,one_hot_x = model(batch_data,ce_loss,optim,true_label)
    
    pred = np.argmax(pred,1)
    batch_label = batch_label.cpu().data.numpy()
    
    preds.extend(pred.tolist())
    labels.extend(batch_label.tolist())
    
    temp_acc = (batch_label == pred).mean()
    acc.append(temp_acc)
    total_batch_acc.append(batch_acc + [temp_acc])
    total_batch_grad.append([0] + batch_grad)
    total_batch_acc_str = '[' + ' '.join(map(lambda x: '%.10f'%x,np.mean(total_batch_acc,axis=0))) + ']'
    total_batch_grad_str = '[' + ' '.join(map(lambda x: '%.10f'%x,np.mean(total_batch_grad,axis=0))) + ']'
    bar.set_description(total_batch_acc_str)
#     print(total_batch_acc_str)
#     print(total_batch_grad_str)
#     print(f"test：{temp_acc:4f}, test mean: {np.mean(acc):4f}")

[0.0074561406 0.0074561406 0.0070175440 0.0061403508 0.0048245615 0.0013157895 0.0004385965 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.000000000

In [16]:
torch.unique(torch.sum(one_hot_x,dim=-1))

tensor([1.], device='cuda:0', grad_fn=<Unique2Backward0>)

In [ ]:
# import pickle

In [ ]:
# with open('/workdir/security/home/junjiehuang2468/paper/results/ember/adv/my_way/PGD_epoch:2_test_acc:0.884008.pt','wb') as fp:
#     pickle.dump(total_batch_acc,fp)

In [ ]:
# import pickle

In [ ]:
# with open('/workdir/security/home/junjiehuang2468/paper/results/ember/my_way/cumulative_gradients_v4_13iter_grad.txt','wb') as fp:
#     pickle.dump(total_batch_grad,fp)